# some sort of labeling object

## use cases

* set of images
* set of ordered images from a video
* set of images (possibly ordered) with bounding-box annotations

## constructor inputs

* output filepath
* list of images
* boolean indicator if ordered
* optional list of bounding boxes


## under-the-hood functions

* convert local to global coords and back
* propagate labels
* estimate homography matrix

## user-facing functions

* next/previous image
* save annotation
* remove annotation


## record in annotations

* patch corner coords (global)
* origin of coordinate system for local coords (for cropping later)


note: can't register images of different shapes. so for different-shaped crops need to find max or min vals for each dimension or something

In [ ]:
import holoviews as hv
import numpy as np
import panel as pn
from PIL import Image

hv.extension('bokeh')
pn.extension()

In [ ]:
def _build_holoviews_annotator(img, xs=None, ys=None):
    x = np.array(img)
    img_hv = hv.RGB(x, bounds=[0, 0, x.shape[1], -1*x.shape[0]])

    # initialize a sequence of 4 points if none provided
    if (xs is None)&(ys is None):
        imshape = x.shape
        a = imshape[1]/2 - imshape[1]/4
        b = imshape[1]/2 + imshape[1]/4
        c = -1*imshape[0]/2 - imshape[0]/4
        d = -1*imshape[0]/2 + imshape[0]/4
        xs = [a,a,b,b]
        ys = [d,c,c,d]

    curve = hv.Curve((xs, ys))
    curve_annotator = hv.annotate.instance()
    fig = img_hv*curve_annotator(curve, annotations={"Label":str})
    return fig, curve_annotator

In [ ]:
def get_homography_transform(im1, im2, num_iterations=1000, pyramid_levels=7):
    """

    """
    reg = kornia.geometry.ImageRegistrator("homography", 
                                       num_iterations=num_iterations,
                                      pyramid_levels=pyramid_levels,
                                       tolerance=1e-8)
    return reg.register(im1, im2)

In [ ]:
def transform_coords(xs, ys, matrix, H, W):
    # turn coords into a tensor with y-axis flipped and normalized to (-1,1) interval
    lab = torch.tensor([ 2*xs/W - 1, -2*ys/H - 1]).permute(1,0).unsqueeze(0).type(torch.float32)
    # transform coords
    transformed_lab = kornia.geometry.homography.transform_points(matrix.inverse(), lab).detach()
    new_xs = (W*transformed_lab[0,:,0].numpy() + W)/2
    new_ys = (H*transformed_lab[0,:,1].numpy() + H)/2
    return new_xs, new_ys

In [ ]:
img = Image.open("/Users/joe/Documents/electricmayhem_tutorials/data/flower2.png")
fig, annotator = _build_holoviews_annotator(img)

In [ ]:
fig

In [ ]:
buttons = {
    "back":pn.widgets.Button(name="back"),
    "next":pn.widgets.Button(name="next"),
    #"dontsave_next":pn.widgets.Button(name="next (dont save)"),
    #"dontprop_next":pn.widgets.Button(name="next (dont propagate)"),
}

keyframe = pn.widgets.Checkbox(name="keyframe", value=False)
progress = pn.indicators.Progress(value=10, width=75)

pn.Row(pn.Column(buttons["back"],
                buttons["next"],
                #buttons["dontsave_next"],
                #buttons["dontprop_next"],
                keyframe,
                progress),
    pn.pane.HoloViews(fig.DynamicMap))

In [ ]:
annotator.annotated.dframe()

In [ ]:
annotator.annotated = hv.Curve(([1,2,3], [4,5,6]))

In [ ]:
type(annotator.annotated)

In [ ]:
#x = np.array(Image.open("/home/joe/Documents/electricmayhem_tutorials/data/flower2.png"))
x = np.array(Image.open("/Users/joe/Documents/electricmayhem_tutorials/data/flower2.png"))
imshape = x.shape
print(imshape)
img = hv.RGB(x, bounds=[0, 0, 320, -240])#.opts(invert_yaxis=True)
img

In [ ]:
imshape = x.shape
a = imshape[1]/2 - imshape[1]/4
b = imshape[1]/2 + imshape[1]/4
c = -1*imshape[0]/2 - imshape[0]/4
d = -1*imshape[0]/2 + imshape[0]/4

curve = hv.Curve(([a,a,b,b], [d,c,c,d]))
curve

In [ ]:
curve_annotator = hv.annotate.instance()

In [ ]:
img*curve_annotator(curve, annotations={"Label":str})

In [ ]:
curve_annotator.annotated.dframe()

In [ ]:
import matplotlib.pyplot as plt
import torch
import kornia.geometry, kornia.augmentation

In [ ]:
#im = np.array(Image.open("/home/joe/Documents/electricmayhem_tutorials/data/flower2.png")).astype(np.float32)/255
im = np.array(Image.open("/Users/joe/Documents/electricmayhem_tutorials/data/flower2.png")).astype(np.float32)/255
imt = torch.tensor(im).permute(2,0,1)[:3,:,:]
print(imt.shape)
plt.imshow(im);

In [ ]:
aug = kornia.augmentation.RandomAffine(scale=(0.5,1.5), translate=0.25, degrees=10)
im1 = aug(imt).detach()
im2 = aug(imt).detach()
plt.subplot(121)
plt.imshow(im1.squeeze(0).permute(1,2,0).numpy())
plt.subplot(122)
plt.imshow(im2.squeeze(0).permute(1,2,0).numpy());

In [ ]:
aug._params

In [ ]:
kornia.geometry.ImageRegistrator.register?

In [ ]:
im1.shape

In [ ]:
#reg = kornia.geometry.ImageRegistrator(num_iterations=1000)
reg = kornia.geometry.ImageRegistrator("homography", 
                                       num_iterations=1000,
                                      pyramid_levels=7, # 5
                                       tolerance=1e-8
                                      )

In [ ]:
%%time
model = reg.register(im1[:,:,:200,:], im2)

In [ ]:
model

In [ ]:
model.shape

In [ ]:
warped = kornia.geometry.homography_warp(im1, model, (240,320), normalized_homography=True)

In [ ]:
plt.subplot(121)
plt.imshow(im1.squeeze(0).permute(1,2,0).numpy())
plt.title("original")
plt.subplot(122)
plt.title("warped")
plt.imshow(warped.squeeze(0).permute(1,2,0).detach().numpy());

In [ ]:
plt.subplot(121)
plt.title("warped")
plt.imshow(warped.squeeze(0).permute(1,2,0).detach().numpy())
plt.subplot(122)
plt.title("target")
plt.imshow(im2.squeeze(0).permute(1,2,0).numpy());

In [ ]:
img = hv.RGB(im1.squeeze(0).permute(1,2,0).numpy(), bounds=[0, 0, 320, -240])#.opts(invert_yaxis=True)
imshape = x.shape
a = imshape[1]/2 - imshape[1]/4
b = imshape[1]/2 + imshape[1]/4
c = -1*imshape[0]/2 - imshape[0]/4
d = -1*imshape[0]/2 + imshape[0]/4
curve = hv.Curve(([a,a,b,b], [d,c,c,d]))
curve_annotator = hv.annotate.instance()
img*curve_annotator(curve, annotations={"Label":str})

In [ ]:
curve_annotator.annotated.dframe()

In [ ]:
# directly put pixels through
df = curve_annotator.annotated.dframe()
lab = torch.tensor([-1*df.y.values, df.x.values]).permute(1,0).unsqueeze(0).type(torch.float32)
transformed_lab = kornia.geometry.homography.transform_points(model, lab).detach()

plt.subplot(131)
plt.imshow(im1.squeeze(0).permute(1,2,0).numpy())
plt.plot(lab[0,:,1].numpy(), lab[0,:,0].numpy())
plt.subplot(132)
plt.title("warped")
plt.imshow(warped.squeeze(0).permute(1,2,0).detach().numpy())
plt.plot(lab[0,:,1].numpy(), lab[0,:,0].numpy())
plt.plot(transformed_lab[0,:,1].numpy(), transformed_lab[0,:,0].numpy())
plt.subplot(133)
plt.title("target")
plt.imshow(im2.squeeze(0).permute(1,2,0).numpy())
plt.plot(transformed_lab[0,:,1].numpy(), transformed_lab[0,:,0].numpy());

In [ ]:
# attempt conversion to normalized coordinates to (-1,1) first
df = curve_annotator.annotated.dframe()
lab = torch.tensor([-1*df.y.values/120-1, df.x.values/160-1]).permute(1,0).unsqueeze(0).type(torch.float32)
transformed_lab = kornia.geometry.homography.transform_points(model, lab).detach()

plt.subplot(131)
plt.imshow(im1.squeeze(0).permute(1,2,0).numpy())
plt.plot(160*lab[0,:,1].numpy()+160, 120*lab[0,:,0].numpy()+120)
plt.subplot(132)
plt.title("warped")
plt.imshow(warped.squeeze(0).permute(1,2,0).detach().numpy())
plt.plot(160*lab[0,:,1].numpy()+160, 120*lab[0,:,0].numpy()+120)
plt.plot(160*transformed_lab[0,:,1].numpy()+160, 120*transformed_lab[0,:,0].numpy()+120)
plt.subplot(133)
plt.title("target")
plt.imshow(im2.squeeze(0).permute(1,2,0).numpy())
plt.plot(160*transformed_lab[0,:,1].numpy()+160, 120*transformed_lab[0,:,0].numpy()+120);

In [ ]:
# attempt conversion to normalized coordinates to (-1,1) first and use inverse homography
# (suggested in issue 849)
df = curve_annotator.annotated.dframe()
lab = torch.tensor([-1*df.y.values/120-1, df.x.values/160-1]).permute(1,0).unsqueeze(0).type(torch.float32)
transformed_lab = kornia.geometry.homography.transform_points(model.inverse(), lab).detach()

plt.subplot(131)
plt.imshow(im1.squeeze(0).permute(1,2,0).numpy())
plt.plot(160*lab[0,:,1].numpy()+160, 120*lab[0,:,0].numpy()+120)
plt.subplot(132)
plt.title("warped")
plt.imshow(warped.squeeze(0).permute(1,2,0).detach().numpy())
plt.plot(160*lab[0,:,1].numpy()+160, 120*lab[0,:,0].numpy()+120)
plt.plot(160*transformed_lab[0,:,1].numpy()+160, 120*transformed_lab[0,:,0].numpy()+120)
plt.subplot(133)
plt.title("target")
plt.imshow(im2.squeeze(0).permute(1,2,0).numpy())
plt.plot(160*transformed_lab[0,:,1].numpy()+160, 120*transformed_lab[0,:,0].numpy()+120);

In [ ]:
# attempt conversion to normalized coordinates to (0,1) first
df = curve_annotator.annotated.dframe()
lab = torch.tensor([-1*df.y.values/240, df.x.values/320]).permute(1,0).unsqueeze(0).type(torch.float32)
transformed_lab = kornia.geometry.homography.transform_points(model, lab).detach()

plt.subplot(131)
plt.imshow(im1.squeeze(0).permute(1,2,0).numpy())
plt.plot(320*lab[0,:,1].numpy(), 240*lab[0,:,0].numpy())
plt.subplot(132)
plt.title("warped")
plt.imshow(warped.squeeze(0).permute(1,2,0).detach().numpy())
plt.plot(320*lab[0,:,1].numpy(), 240*lab[0,:,0].numpy())
plt.plot(320*transformed_lab[0,:,1].numpy(), 240*transformed_lab[0,:,0].numpy())
plt.subplot(133)
plt.title("target")
plt.imshow(im2.squeeze(0).permute(1,2,0).numpy())
plt.plot(320*transformed_lab[0,:,1].numpy(), 240*transformed_lab[0,:,0].numpy());

In [ ]:
# directly put pixels through inverse transform
df = curve_annotator.annotated.dframe()
lab = torch.tensor([-1*df.y.values, df.x.values]).permute(1,0).unsqueeze(0).type(torch.float32)
transformed_lab = kornia.geometry.homography.transform_points(model.inverse(), lab).detach()

plt.subplot(131)
plt.imshow(im1.squeeze(0).permute(1,2,0).numpy())
plt.plot(lab[0,:,1].numpy(), lab[0,:,0].numpy())
plt.subplot(132)
plt.title("warped")
plt.imshow(warped.squeeze(0).permute(1,2,0).detach().numpy())
plt.plot(lab[0,:,1].numpy(), lab[0,:,0].numpy())
plt.plot(transformed_lab[0,:,1].numpy(), transformed_lab[0,:,0].numpy())
plt.subplot(133)
plt.title("target")
plt.imshow(im2.squeeze(0).permute(1,2,0).numpy())
plt.plot(transformed_lab[0,:,1].numpy(), transformed_lab[0,:,0].numpy());

In [ ]:
# directly put pixels through, swapping x and y channels
df = curve_annotator.annotated.dframe()
lab = torch.tensor([df.x.values, -1*df.y.values]).permute(1,0).unsqueeze(0).type(torch.float32)
transformed_lab = kornia.geometry.homography.transform_points(model, lab).detach()

plt.subplot(131)
plt.imshow(im1.squeeze(0).permute(1,2,0).numpy())
plt.plot(lab[0,:,0].numpy(), lab[0,:,1].numpy())
plt.subplot(132)
plt.title("warped")
plt.imshow(warped.squeeze(0).permute(1,2,0).detach().numpy())
plt.plot(lab[0,:,0].numpy(), lab[0,:,1].numpy())
plt.plot(transformed_lab[0,:,0].numpy(), transformed_lab[0,:,1].numpy())
plt.subplot(133)
plt.title("target")
plt.imshow(im2.squeeze(0).permute(1,2,0).numpy())
plt.plot(transformed_lab[0,:,0].numpy(), transformed_lab[0,:,1].numpy());

In [ ]:
# attempt conversion to normalized coordinates to (-1,1) first and use inverse homography
# (suggested in issue 849)
# but this time also swap x and y channels
df = curve_annotator.annotated.dframe()
lab = torch.tensor([df.x.values/160-1, -1*df.y.values/120-1]).permute(1,0).unsqueeze(0).type(torch.float32)
transformed_lab = kornia.geometry.homography.transform_points(model.inverse(), lab).detach()

plt.subplot(131)
plt.imshow(im1.squeeze(0).permute(1,2,0).numpy())
plt.plot(160*lab[0,:,0].numpy()+160, 120*lab[0,:,1].numpy()+120)
plt.subplot(132)
plt.title("warped")
plt.imshow(warped.squeeze(0).permute(1,2,0).detach().numpy())
plt.plot(160*lab[0,:,0].numpy()+160, 120*lab[0,:,1].numpy()+120)
plt.plot(160*transformed_lab[0,:,0].numpy()+160, 120*transformed_lab[0,:,1].numpy()+120)
plt.subplot(133)
plt.title("target")
plt.imshow(im2.squeeze(0).permute(1,2,0).numpy())
plt.plot(160*transformed_lab[0,:,0].numpy()+160, 120*transformed_lab[0,:,1].numpy()+120);

In [ ]:
def transform_coords(xs, ys, matrix, H, W):
    # turn coords into a tensor with y-axis flipped and normalized to (-1,1) interval
    lab = torch.tensor([ 2*xs/W - 1, -2*ys/H - 1]).permute(1,0).unsqueeze(0).type(torch.float32)
    # transform coords
    transformed_lab = kornia.geometry.homography.transform_points(matrix.inverse(), lab).detach()
    new_xs = (W*transformed_lab[0,:,0].numpy() + W)/2
    new_ys = (H*transformed_lab[0,:,1].numpy() + H)/2
    return new_xs, new_ys